# Curso de Optimización (DEMAT)
## Tarea 9

| Descripción:                         | Fechas               |
|--------------------------------------|----------------------|
| Fecha de publicación del documento:  | **Abril 28, 2022**   |
| Fecha límite de entrega de la tarea: | **Mayo   8, 2022**   |


### Indicaciones

- Envie el notebook que contenga los códigos y las pruebas realizadas de cada ejercicio.
- Si se requiren algunos scripts adicionales para poder reproducir las pruebas,
  agreguelos en un ZIP junto con el notebook.
- Genere un PDF del notebook y envielo por separado.

---

## Ejercicio 0 (0 puntos)

- Empezar a buscar un tema para el proyecto final del curso.
- En esta tarea no hay que poner la descripción del proyecto.
  Sólo buscar el tema y tenerlo listo para su entrega en la
  siguiente tarea.
- La fecha límite para mandar la descripción del proyecto
  se tiene que mandar es el domingo 17 de mayo. 



```



```
---


## Ejercicio 1. (2 puntos)

Programar las siguientes funciones y sus gradientes, de modo que dependan de la dimensión $n$ de la variable $\mathbf{x}$:


- Función "Tridiagonal 1" generalizada

$$  f(x) = \sum_{i=1}^{n-1} (x_i + x_{i+1} - 3)^2 + (x_i - x_{i+1} + 1)^4  $$


- Función generalizada de Rosenbrock

$$  f(x) = \sum_{i=1}^{n-1} 100(x_{i+1} - x_i^2)^2 + (1 - x_{i} )^2  $$


In [1]:
import numpy as np

#  Implementación de la función Tridiagonal y su gradiente
def tridiagonal(x):
    x1 = x[:-1]
    x2 = x[1:]

    x3 = (x1 + x2 - 3)**2 + (x1 - x2 + 1)**4
    return np.sum(x3)

def g_tridiagonal(x):
    n = x.shape[0]
    
    x1 = x[:-1]
    x2 = x[1:]
    
    d1 = 2 * (x1 + x2 - 3) + 4 * (x1 - x2 + 1)**3
    d2 = 2 * (x1 + x2 - 3) - 4 * (x1 - x2 + 1)**3
    
    d = np.zeros((n,))
    d[0] = d1[0]
    d[1:-1] = d1[1:] + d2[:-1]
    d[-1] = d2[-1]
    
    return d

In [2]:
#  Implementación de la función generalizada de Rosenbrock y su gradiente

def rosenbrock(x):
    x1 = x[:-1]
    x2 = x[1:]

    x3 = 100 * (x2 - x1**2)**2 + (1-x1)**2
    return np.sum(x3)
    
def g_rosenbrock(x):
    n = x.shape[0]
    
    x1 = x[:-1]
    x2 = x[1:]
    
    d1 = -400 * (x2 - x1**2) * x1 - 2 * (1 - x1)
    d2 = 200 * (x2 - x1**2)
    
    d = np.zeros((n,))
    d[0] = d1[0]
    d[1:-1] = d1[1:] + d2[:-1]
    d[-1] = d2[-1]
    
    return d


```


```
---


## Ejercicio 1 (8 puntos)

Programar y probar el método BFGS modificado.


1. Programar el algoritmo descrito en la diapositiva 16 de la clase 23.
   Agregue una variable $res$ que indique si el algoritmo terminó
   porque se cumplió que la magnitud del gradiente es menor que la toleracia
   dada.
2. Probar el algoritmo con las funciones del Ejercicio 1
   con la matriz $H_0$ como la matriz identidad y el 
   punto inicial $x_0$ como:

- La función generalizada de Rosenbrock: 

$$ x_0 = (-1.2, 1, -1.2, 1, ..., -1.2, 1) \in \mathbb{R}^n$$

- La función Tridiagonal 1 generalizada: 

$$ x_0 = (2,2, ..., 2) \in \mathbb{R}^n $$
  
  Pruebe el algoritmo con la dimensión $n=2, 10 , 100$.

3. Fije el número de iteraciones máximas a $N=50000$, 
   y la tolerancia $\tau = \epsilon_m^{1/3}$, donde $\epsilon_m$
   es el épsilon máquina, para terminar las iteraciones 
   si la magnitud del gradiente es menor que $\tau$.
   En cada caso, imprima los siguiente datos:
   
- $n$,
- $f(x_0)$, 
- Usando la variable $res$, imprima un mensaje que indique si
  el algoritmo convergió,
- el  número $k$ de iteraciones realizadas,
- $f(x_k)$,
- la norma del vector $\nabla f_k$, y
- las primeras y últimas 4 entradas del punto $x_k$ que devuelve el algoritmo.
  

### Solución:

In [3]:
def backtracking(f, xk, pk, p, beta):
    alpha = 1
    fk = f(xk)
    while True:
        if f(xk + alpha * pk) <= fk - beta*alpha*(pk.T @ pk):
            return alpha
        alpha = p * alpha

def BFGS(fun, grad, x0, H0, maxN, tol):
    res = 0
    n = x0.shape[0]
    Hk = H0
    
    x_new = x0
    g_new = grad(x_new)
    
    for k in range(maxN):
        xk = x_new
        gk = g_new
        #print(xk, gk)
        
        if np.linalg.norm(gk) <= tol:
            res = 1
            break
        
        pk = -Hk @ gk
        
        if pk.T @ gk > 0:
            lamb1 = 10**(-5) + (pk.T @ gk) / (gk.T @ gk)
            Hk = Hk + lamb1 * np.eye(n)
            pk = pk - lamb1 * gk
        
        alpha = backtracking(fun, xk, pk, 0.8, 0.0001)
        x_new = xk + alpha * pk
        g_new = grad(x_new)

        sk = x_new - xk
        yk = g_new - gk

        lamb2 = 10**(-5) - (yk.T @ sk) / (sk.T @ sk)
        if yk.T @ sk <= 0:
            Hk = Hk + lamb2 * np.eye(n)
        else:
            rho = 1 / (yk.T @ sk)
            Hk = (np.eye(n) - rho * sk @ yk.T) @ Hk @ (np.eye(n) - rho * yk @ sk.T) + rho * sk @ sk.T
        
        if np.abs(g_new.T @ gk) > 0.2 * np.linalg.norm(g_new)**2:
            # Reinicio
            Hk = np.eye(n)
                
    return xk, gk, k, res

In [4]:
# Pruebas realizadas 

def short(x):
    if x.shape[0] <= 8:
        print('(', *x, ')')
        return
    y = np.ones((8,))
    y[:4] = x[:4]
    y[4:] = x[-4:]
    print('(', *x[:4], '...', *x[-4:], ')' )

def test_BFGS(n, fun, grad, x0, maxN, tol, mess):
    print(f"Test BFGS | {mess}")
    print(f'n = {n}')
    print(f'f(x0) = {fun(x0)}')
    
    xk, gk, k, res = BFGS(fun, grad, x0, np.eye(n), maxN, tol)
    
    print('xk = ', end='')
    short(xk)
    print(f'||gk|| = {np.linalg.norm(gk)}')
    print(f'f(xk) = {fun(xk)}')
    print(f'k = {k}')
    if res == 1:
        print(f'El algoritmo convergio')
    else:
        print('El algoritmo no convergio')
    print()


EPS_M = np.finfo(float).eps
MAX_N = 50000
TOL = EPS_M ** (1/3)

NN = (2, 10, 100)

for n in NN:
    x0 = np.ones((n,))
    x0 = x0 * 2
    test_BFGS(n, tridiagonal, g_tridiagonal, x0, MAX_N, TOL, 'Tridiagonal')
print()

for n in NN:
    x0 = np.ones((n,))
    x0[::2] = -1.2
    test_BFGS(n, rosenbrock, g_rosenbrock, x0, MAX_N, TOL, 'Rosenbrock')

Test BFGS | Tridiagonal
n = 2
f(x0) = 2.0
xk = ( 0.9968641547054268 2.003137883035577 )
||gk|| = 5.930456824986014e-06
f(xk) = 1.5533357873849146e-09
k = 3163
El algoritmo convergio

Test BFGS | Tridiagonal
n = 10
f(x0) = 18.0
xk = ( 1.0246464559571948 1.3436102047625293 1.4389090298938394 1.476453221824012 ... 1.523546778175988 1.5610909701061606 1.6563897952374707 1.9753535440428052 )
||gk|| = 5.7363218184731905e-06
f(xk) = 7.211216703291889
k = 258
El algoritmo convergio

Test BFGS | Tridiagonal
n = 100
f(x0) = 198.0
xk = ( 1.0244815803139797 1.3432606516996888 1.4381179955589474 1.4745905383926357 ... 1.5254094541662702 1.5618819963385921 1.65673933312725 1.9755183835721504 )
||gk|| = 4.4551032117350855e-06
f(xk) = 97.21030748599117
k = 62
El algoritmo convergio


Test BFGS | Rosenbrock
n = 2
f(x0) = 24.199999999999996
xk = ( 1.0000029421689873 1.0000058839580541 )
||gk|| = 6.040269183391008e-06
f(xk) = 8.656373449210375e-12
k = 16816
El algoritmo convergio

Test BFGS | Rosenbrock
